## Explicabilidade

Este trabalho tem como objetivo o treinamento de modelos explicáveis para o problema de análise de crimes segundo características socio-econômicas nas regiões de São Paulo. Aqui, retornamos ao uso de todas as variáveis socio-econômicas para obtenção de explicações completas acerca da origem e desenvolvimento criminal nas regiões. No entanto, utilizaremos uma arquitetura de modelos diferente das utilizadas nos trabalhos anteriores.

O desafio de prever crimes numa região poderia ser modelada como uma regressão para prever a distribuição dos boletins de ocorrência, por exemplo, no tempo. No entanto, regiões com baixo índice de criminalidade não possuem amostras de boletins de ocorrência capazes de se extrair uma distribuição que modele os BOs de forma satisfatória, ou até mesmo fiel. A forma que encontramos para minimizar este problema anteriormente foi de modelar o problema como um problema de classificação em regiões perigosas ou não, baseado apenas na quantidade de BOs registrados em dada região. No entanto, este tipo de redução carrega inúmeros vieses e não é justo para tomar qualquer tipo de decisão, principalmente devido a ambiguidade que um hard threshold causa ao diferenciar amostras cujo padrão é semelhante, mas o ground-truth induzido não é, por exemplo, classificar como perigoso regiões com mais de 1 BOs por dia em média classifica regiões com 0.99 BOs por dia como seguros.

Pensando nisso e numa forma de se aprimorar o problema em si de forma que sua saída seja mais fácil de ser explicada, utilizamos um modelo híbrido de Regressão Inflada em zero. Esta abordagem melhora a capacidade de explicação ao separar regiões perigosas de não perigosas (zero e não-zero) e então obter um modelo de regressão para regiões perigosas, o que garante modelos capazes de quantificar a incidência criminal e não apenas sua existência.

### Revisão Bibliográfica

[Curiel et. al](https://link.springer.com/article/10.1007/s10940-017-9354-9) argumenta que a distribuição de ocorrências criminais ocorre segundo uma distribuição de Poisson-Binomial, onde cada indivíduo é selecionado por uma Binomial e então a ocorrência é modelada por uma distribuição de Poisson. Deste modo, escolhemos tratar nosso problema de forma que cada região é tratada como uma amostra de uma distribuição de Poisson que tenta prever a quantidade de BOs relatados naquela região. Uma regressão de Poisson utiliza um aproximador de esperança dado por

$$\mathbb{E}[y \,|\, x] = \exp(\theta^Tx)$$

Que induz uma distribuição de probabilidade condicional de Poisson

$$p_\theta(y \,|\, x) = \frac{\exp(y \cdot \theta^Tx)}{y!} \exp(-e^{\theta^Tx})$$

Os parâmetros $\theta$ são então otimizados de forma a maximizar a verossimilhança dos dados $\mathcal{D} = \{x^{(i)}, y^{(i)}\}_{i=1}^N$, 

$$\theta = \argmax_{\theta}\prod_{i=1}^N p_\theta(y_i \,|\, x_i)$$

Como trata-se de um problema de regressão em que 

In [1]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import PoissonRegressor
from sklearn.pipeline import Pipeline, make_pipeline
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme('notebook', 'whitegrid', 'Set1')

In [138]:
data = pd.read_csv('../dataset/Padrão Urbano/PU.csv', sep=';')

selected_columns = set(''.join([i for i in column if not i.isdigit()]) for column in data.columns)
selected_columns = [f"{column}10"if len(column) == 3 else column for column in selected_columns]

X = data[selected_columns].drop(columns=['SETTT'])

categ_columns = ['HOMIC', 'ARISC', 'PMANC', 'EXURB', 'AGL10', 'DEN10', 'CLUSTER']
log_columns = ['VIAGEM', 'POP10']

X['CLUSTER'] = OrdinalEncoder().fit_transform(X[['CLUSTER']])

preprocess = ColumnTransformer([
    ("log_numeric", make_pipeline(FunctionTransformer(np.log1p).set_output(transform="pandas"), StandardScaler().set_output(transform="pandas")), log_columns),
    ("onehot_categ", OneHotEncoder(sparse_output=False).set_output(transform="pandas"), categ_columns)
], remainder = StandardScaler().set_output(transform="pandas")).set_output(transform="pandas")

X.head(5)

/home/danielgardin/.local/lib/python3.8/site-packages/sklearn/preprocessing/_function_transformer.py:340: UserWarning: With transform="pandas", `func` should return a DataFrame to follow the set_output API.
  warnings.warn(


,DEN10,CMU10,ARISC,ESG10,DPP10,PMANC,CLUSTER,AGU10,VER10,PJM10,HOMIC,LIX10,CAR10,AGL10,POP10,CAL10,VIAGEM,EXURB,TDESL,DPI10
0,2,0.3619,0,1.0,0.9925,0,2.0,1.0,0.0410,0.0558,1,0.9962,0.0224,0,806,0.9925,1428.9,2,29.8,0.0075
1,2,0.3268,0,1.0,1.0000,0,0.0,1.0,0.0000,0.0701,1,1.0000,0.0163,0,913,0.9706,1554.2,1,21.7,0.0000
2,2,0.4392,0,1.0,1.0000,0,2.0,1.0,0.0370,0.0560,1,1.0000,0.0106,0,625,0.9788,1471.5,2,25.8,0.0000
3,2,0.5414,0,1.0,1.0000,0,1.0,1.0,0.1271,0.0752,2,1.0000,0.0055,0,572,0.9834,1404.8,2,27.7,0.0000
4,2,0.4458,0,1.0,1.0000,0,1.0,1.0,0.0500,0.0809,2,1.0000,0.0000,0,754,0.9667,1492.0,2,28.6,0.0000


In [139]:
crimes = pd.read_csv('../dataset/Crimes/Listagem_Geral.csv', index_col=0)

crimes["DATA"] = pd.to_datetime(crimes["DATA"])

counts = crimes['SETOR'].value_counts()

y = data['SETTT'].apply(lambda setor : counts.get(setor, 0)).to_numpy()

In [140]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [141]:
frac = 0.6

n_aug_samples = int(len(X_train) * frac)

idx = np.random.randint(0, len(X_train), size=n_aug_samples)

X_aug = X_train.iloc[idx].copy()

X_aug["CAR10"] = X_train["CAR10"].sample(n=n_aug_samples, replace=True, random_state=21).to_numpy()

y_aug = y_train[idx]

X_aug = pd.concat([X_train, X_aug])
y_aug = np.hstack([y_train, y_aug])

In [142]:
from sklearn.model_selection import KFold
from tqdm import tqdm

def evaluate(models, X, y, metrics, cv=20, df=None, figname=None):
    if df is None:
        df = pd.DataFrame(columns=metrics.keys())
    
    results = {}
    for name, model in models.items():
        scores = {metric : [] for metric in metrics}

        for train_idx, val_idx in tqdm(KFold(n_splits=cv).split(X, y)):
            model.fit(X.iloc[train_idx], y[train_idx])

            for metric_name, metric in metrics.items():
                scores[metric_name].append(metric(model, X.iloc[val_idx], y[val_idx]))

        results[name] = scores
        df.loc[name, :] = scores

        model.fit(X_train, y_train)

    fig = plt.figure(figsize=(10,5))
    ax = fig.add_subplot(111)
    ax.set_title('Algorithm Comparison')

    n_metrics = len(metrics)
    n_models  = len(models)

    names = list(models.keys())

    spacing = 0.1
    delta   = (1 - spacing)*(1 - 1/n_metrics)/2

    positions = np.hstack([np.linspace(i - delta, i + delta, n_metrics) for i in range(n_models)])
    boxes = plt.boxplot(np.array([model_result[metric] for model_result in results.values() for metric in metrics]).T, positions=positions, widths=1.5*delta/(n_metrics-1), patch_artist=True)

    ax.xaxis.set_major_formatter(lambda _, i : names[i//n_metrics] if i%n_metrics==1 else '')
    
    for i in range(n_metrics * n_models):
        color = sns.color_palette(n_colors=n_metrics)[i%n_metrics]

        boxes['boxes'][i].set_color((1, 1, 1, 0.4))
        boxes['boxes'][i].set(facecolor=color, alpha=0.8)
        boxes['medians'][i].set_color((0, 0, 0, 0.4))
        boxes['whiskers'][2*i].set_color(color)
        boxes['whiskers'][2*i+1].set_color(color)
        boxes['caps'][2*i].set_color(color)
        boxes['caps'][2*i+1].set_color(color)
        boxes['fliers'][i].set_color(color)
        boxes['fliers'][i].set_color(color)

    for i, metric in enumerate(metrics):
        boxes['boxes'][i].set_label(metric)

    plt.ylabel("Score")
    plt.legend(frameon=True, bbox_to_anchor=(1.1, 1.05))
    plt.ylim(-100, 1)

    if figname is not None:
        plt.savefig(f"{figname}.png")

    plt.show()

    return df

def style(df, format_table):

    new_df = df.applymap(lambda x : f"{np.mean(x)} $\pm$ {np.std(x)}")
    return new_df.style.highlight_max().format(format_table,)


In [143]:
from sklearn.metrics import get_scorer, pairwise_distances, balanced_accuracy_score, get_scorer_names

def consistency_score(estimator, X, y, k_neigh=5, metric="euclidean"):
    distances = pairwise_distances(X, metric=metric)

    y_pred = estimator.predict(X)

    y_neigh = y_pred[np.argsort(distances, axis=1)[:, 1:k_neigh+1]].mean(axis=1)

    return 1 - np.abs(y - y_neigh).mean()

def regression_accuracy(estimator, X, y, lim=1):
    y_pred = estimator.predict(X)

    y_pred = np.where(y_pred > lim, 1, 0)
    y_norm = np.where(y > lim, 1, 0)

    return balanced_accuracy_score(y_norm, y_pred)


metrics = {
    "MAE" : get_scorer("neg_mean_absolute_error"),
    "Bal Acc" : regression_accuracy,
    "Consistency" : consistency_score
}

format_table = {
    "MAE" : "{:3f}",
    "Bal Acc" : "{:.2%}",
    "Consistency" : "{:3f}"
}

In [144]:
def round_down(n):
    if n == 0: return 0

    base = 10**np.floor(np.log10(np.abs(n)))
    return np.floor(n / base) * base

def round_up(n):
    if n == 0: return 0
    
    base = 10**np.floor(np.log10(np.abs(n)))
    return np.ceil(n / base) * base

def align_axes(main_ax:plt.Axes, twin_ax:plt.Axes, new_ticks=None):
    l, u = main_ax.get_ylim()
    ticks = main_ax.get_yticks()
    ticks = ticks[(ticks >= l) & (ticks <= u)]

    ticks_given = False
    if new_ticks is None:
        lower, upper = twin_ax.get_ylim()

        max_delta = (upper - lower)/(len(ticks) - 1)

        delta     = round_up(max_delta)

        tick_min  = round_down(lower)

        tick_max  = tick_min + delta * (len(ticks) - 1)

    elif len(new_ticks) == 2:
        tick_min, tick_max = new_ticks

    elif len(new_ticks) == len(ticks): 
        tick_min, tick_max = new_ticks[0], new_ticks[-1]
        ticks_given = True

        if not isinstance(new_ticks, np.ndarray):
            new_ticks = np.array(new_ticks)
    else:
        raise TypeError("Length of the target ticks do not match with the ticks on main axes.")

    bminusa = (tick_max - tick_min) * (u - l)/(ticks[-1] - ticks[0])

    a = tick_max - (ticks[-1] - l)/(u - l) * bminusa
    b = bminusa + a

    if not ticks_given:
        new_ticks = np.linspace(tick_min, tick_max, len(ticks))

    twin_ax.set_yticks(new_ticks)
    twin_ax.set_ylim(a, b)

def plot_metrics(model, X, y, interval=(0, 100)):
    fig, ax = plt.subplots(ncols=2, figsize=(16, 5), dpi=200)

    liminf, limsup = interval

    n ,bins, patches = ax[0].hist(y, bins=np.linspace(liminf, limsup, 100), alpha=.5, label='Ground truth')
    y_pred = model.predict(X)
    percent = np.count_nonzero((y_pred >= liminf) & (y_pred <= limsup))/len(y_pred)
    
    ax[0].hist(y_pred, bins=bins, alpha=.5, label="Predicted")
    ax[0].set_xlim(*interval)
    ax[0].set_title("Distribution")
    ax[0].set_xlabel('Number of reports')
    ax[0].set_ylabel('Count')
    ax[0].legend()

    twin_ax = ax[0].twinx()
    cuts = np.arange(liminf, limsup)

    bal_acc = [regression_accuracy(model, X, y, cut) for cut in cuts]

    twin_ax.plot(cuts, bal_acc, label='Balanced accuracy')

    align_axes(ax[0], twin_ax, (0, 1))

    res = y - y_pred

    liminf = min(np.quantile(res, percent), np.quantile(res, 1 - percent))
    limsup = max(np.quantile(res, percent), np.quantile(res, 1 - percent))

    ax[1].hist(res, bins=np.linspace(liminf, limsup, 100), alpha=.7)
    ax[1].set_title("Residual distribution")
    ax[1].set_xlim(liminf, limsup)


    plt.show()

### Modelos interpretáveis

In [146]:
from sklearn.linear_model import PoissonRegressor, QuantileRegressor, LinearRegression

X_interp = preprocess.fit_transform(X_train)

linear_model =  LinearRegression()

poisson_model = PoissonRegressor(solver="newton-cholesky", alpha=1e-4)

quantile_model = QuantileRegressor(quantile=0.4, alpha=1e-5, solver='highs')


models = {
    "Linear Regressor"   : linear_model,
    "Poisson Regressor"  : poisson_model,
    "Quantile Regressor" : quantile_model
}

In [100]:
results = evaluate(models, X_train, y_train, metrics, cv=10)

10it [00:16,  1.69s/it]
4it [01:21, 20.43s/it]


KeyboardInterrupt: 

In [87]:
results.applymap(lambda x : f"{np.mean(x) : .3f} $\pm$ {np.std(x) :.3f}")

,MAE,Bal Acc,Consistency
Poisson Regressor,-39.962 $\pm$ 1.288,0.503 $\pm$ 0.001,-54.626 $\pm$ 2.378
Quantile Regressor,-4548.109 $\pm$ 9014.024,0.642 $\pm$ 0.020,-47.771 $\pm$ 2.486
Poisson GBTree,-37.585 $\pm$ 2.075,0.567 $\pm$ 0.016,-47.899 $\pm$ 2.490


In [149]:
quantile_model = QuantileRegressor(quantile=0.1, alpha=1e-4, solver='highs').fit(X_interp, y_train)

plot_metrics(quantile_model, X_interp, y_train)

### Zero-Inflated Models

In [11]:
from sklego.meta import ZeroInflatedRegressor
from sklearn.linear_model import LinearRegression

zero_inflated_models = {
    "Poisson ZIR" : Pipeline([
        ('preprocess', preprocess),
        ('model', ZeroInflatedRegressor(
            LinearRegression(alpha=1e-6), PoissonRegressor(solver="newton-cholesky", alpha=1e-6))
        )
        ]),
    "Quantile ZIR" : Pipeline([
        ('preprocess', preprocess),
        ('model', ZeroInflatedRegressor(
            LinearRegression(alpha=1e-6), PoissonRegressor(solver="newton-cholesky", alpha=1e-6))
        )
        ])

    "boost"
}

SyntaxError: invalid syntax (180656819.py, line 18)

### Modelos não-interpretáveis

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor


poisson_tree = HistGradientBoostingRegressor(loss="quantile", max_leaf_nodes=128, quantile=0.4)